<a href="https://colab.research.google.com/github/marcelofschiavo/ds-cookbook/blob/main/07_ML_Clustering_(Nao_Supervisionado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Conceitos: Encontrar grupos ocultos (Personas).

"Receitas" (Códigos):

A etapa obrigatória: StandardScaler.

Como achar o 'K' ideal: Método do Cotovelo (Elbow Method).

O modelo: KMeans.

Como visualizar: PCA para reduzir para 2D.

Como interpretar: Agrupar por cluster e calcular a média (.groupby('cluster').mean()).